In [7]:
# %matplotlib inline
# %matplotlib widget
%matplotlib ipympl
%load_ext autoreload
%autoreload 2

import ipywidgets as wdg  # Using the ipython notebook widgets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import sys
import getopt
import numpy as np
import os
from glob import glob
import piexif
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import *
from keras.layers import *
from TrackNet3 import TrackNet3
import keras.backend as K
from keras import optimizers
import tensorflow as tf
import cv2
from os.path import isfile, join
from PIL import Image
import time
BATCH_SIZE=1
HEIGHT=288
WIDTH=512
sigma=2.5
mag=1

In [9]:
def genHeatMap(w, h, cx, cy, r, mag):
	if cx < 0 or cy < 0:
		return np.zeros((h, w))
	x, y = np.meshgrid(np.linspace(1, w, w), np.linspace(1, h, h))
	heatmap = ((y - (cy + 1))**2) + ((x - (cx + 1))**2)
	heatmap[heatmap <= r**2] = 1
	heatmap[heatmap > r**2] = 0
	return heatmap*mag

#time: in milliseconds
def custom_time(time):
	remain = int(time / 1000)
	ms = (time / 1000) - remain
	s = remain % 60
	s += ms
	remain = int(remain / 60)
	m = remain % 60
	remain = int(remain / 60)
	h = remain
	#Generate custom time string
	cts = ''
	if len(str(h)) >= 2:
		cts += str(h)
	else:
		for i in range(2 - len(str(h))):
			cts += '0'
		cts += str(h)
	
	cts += ':'

	if len(str(m)) >= 2:
		cts += str(m)
	else:
		for i in range(2 - len(str(m))):
			cts += '0'
		cts += str(m)

	cts += ':'

	if len(str(int(s))) == 1:
		cts += '0'
	cts += str(s)

	return cts

In [10]:
# A simple one-euro filter
import math

def smoothing_factor(t_e, cutoff):
    r = 2 * math.pi * cutoff * t_e
    return r / (r + 1)

def exponential_smoothing(a, x, x_prev):
    return a * x + (1 - a) * x_prev

class OneEuroFilter:
    def __init__(self, t0, x0, dx0=0.0, min_cutoff=0.004, beta=0.7,
                 d_cutoff=30):
        """Initialize the one euro filter."""
        # The parameters.
        self.min_cutoff = float(min_cutoff)
        self.beta = float(beta)
        self.d_cutoff = float(d_cutoff)
        # Previous values.
        self.x_prev = x0
        if np.array(dx0).shape != np.array(x0).shape:
            self.dx_prev = np.zeros(np.array(x0).shape)
        self.t_prev = t0

    def __call__(self, t, x):
        """Compute the filtered signal."""
        t_e = t - self.t_prev

        # The filtered derivative of the signal.
        a_d = smoothing_factor(t_e, self.d_cutoff)
        dx = (x - self.x_prev) / t_e
        dx_hat = exponential_smoothing(a_d, dx, self.dx_prev)

        # The filtered signal.
        cutoff = self.min_cutoff + self.beta * np.max(np.abs(dx_hat))
        a = smoothing_factor(t_e, cutoff)
        x_hat = exponential_smoothing(a, x, self.x_prev)
        
        # Memorize the previous values.
        self.x_prev = np.array(x_hat)
        self.dx_prev = np.array(dx_hat)
        self.t_prev = t

        return x_hat

In [14]:
# videoName = '/home/code-base/scratch_space/data/match1/rally_video/1_02_00.mp4'
# videoName = '/home/code-base/scratch_space/data/match24/rally_video/1_00_01.mp4'
videoName = '/home/code-base/user_space/ai-badminton/notebooks/data/anthony-ginting-beat-kento-momota-anthony-ginting-vs-kento-momota-fo-2019-hlzrivktbyo.mp4'
load_weights = 'model906_30'

#Loss function
def custom_loss(y_true, y_pred):
    loss = (-1)*(K.square(1 - y_pred) * y_true * K.log(K.clip(y_pred, K.epsilon(), 1)) + K.square(y_pred) * (1 - y_true) * K.log(K.clip(1 - y_pred, K.epsilon(), 1)))
    return K.mean(loss)

model = load_model(load_weights, custom_objects={'custom_loss':custom_loss})


def predict_heatmaps(videoName, outName):
    print('Beginning predicting......')
    start = time.time()

    cap = cv2.VideoCapture(videoName)

    success, image1 = cap.read()
    frame_time1 = custom_time(cap.get(cv2.CAP_PROP_POS_MSEC))
    success, image2 = cap.read()
    frame_time2 = custom_time(cap.get(cv2.CAP_PROP_POS_MSEC))
    success, image3 = cap.read()
    frame_time3 = custom_time(cap.get(cv2.CAP_PROP_POS_MSEC))

    ratio = image1.shape[0] / HEIGHT

    size = (int(WIDTH*ratio), int(HEIGHT*ratio))
    fps = 30

    count = 0

    print('About to begin prediction...')

    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    from tqdm.notebook import tqdm
    pbar = tqdm(total=total)

    predictions = []
    while success:
        unit = []
        #Adjust BGR format (cv2) to RGB format (PIL)
        x1 = image1[...,::-1]
        x2 = image2[...,::-1]
        x3 = image3[...,::-1]
        #Convert np arrays to PIL images
        x1 = array_to_img(x1)
        x2 = array_to_img(x2)
        x3 = array_to_img(x3)
        #Resize the images
        x1 = x1.resize(size = (WIDTH, HEIGHT))
        x2 = x2.resize(size = (WIDTH, HEIGHT))
        x3 = x3.resize(size = (WIDTH, HEIGHT))
        #Convert images to np arrays and adjust to channels first
        x1 = np.moveaxis(img_to_array(x1), -1, 0)
        x2 = np.moveaxis(img_to_array(x2), -1, 0)
        x3 = np.moveaxis(img_to_array(x3), -1, 0)
        #Create data
        unit.append(x1[0])
        unit.append(x1[1])
        unit.append(x1[2])
        unit.append(x2[0])
        unit.append(x2[1])
        unit.append(x2[2])
        unit.append(x3[0])
        unit.append(x3[1])
        unit.append(x3[2])
        unit=np.asarray(unit)    
        unit = unit.reshape((1, 9, HEIGHT, WIDTH))
        unit = unit.astype('float32')
        unit /= 255
        y_pred = model.predict(unit, batch_size=BATCH_SIZE)
        h_pred = y_pred[0]
    #     y_pred = y_pred > 0.5
    #     y_pred = y_pred.astype('float32')
    #     h_pred = y_pred[0]*255
    #     h_pred = h_pred.astype('uint8')
        for i in range(3):
            if i == 0:
                frame_time = frame_time1
                image = image1
            elif i == 1:
                frame_time = frame_time2
                image = image2
            elif i == 2:
                frame_time = frame_time3    
                image = image3

            pred = (h_pred[i] * 255).astype('uint8')
            predictions.append(pred)
            count += 1
            continue
            base = h_pred[i]
    #         if count == 0:
    #             oe_filter = OneEuroFilter(0, h_pred[i])
    #         else:
    #             base = oe_filter(count * 1./fps, h_pred[i])
            rounded_pred = (base.copy() > 0.25) * 255
            rounded_pred = rounded_pred.astype('uint8')

            (cnts, _) = cv2.findContours(rounded_pred, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            rects = [cv2.boundingRect(ctr) for ctr in cnts]
            get_area = lambda rect: np.sum(base[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]])
            max_area_idx = -1
            max_area = 0
            for j in range(len(rects)):
                area = get_area(rects[j]) / (rects[j][2] * rects[j][3])
                if area > max_area:
                    max_area_idx = j
                    max_area = area

            if max_area_idx < 0:
                f.write(str(count)+',0,0,0,'+frame_time+'\n')
                out.write(image)
            else:
                target = rects[max_area_idx]
                area = rects[max_area_idx][2] * rects[max_area_idx][3]
                (cx_pred, cy_pred) = (int(ratio*(target[0] + target[2] / 2)), int(ratio*(target[1] + target[3] / 2)))

                f.write(str(count)+',1,'+str(cx_pred)+','+str(cy_pred)+','+frame_time+'\n')
                image_cp = np.copy(image)
                cv2.circle(image_cp, (cx_pred, cy_pred), int(area**0.5), (0,0,255), -1)
                out.write(image_cp)
    #             image_cp = cv2.resize(base, size)
    #             image_cp = np.stack((image_cp,)*3, axis=-1)
    #             out_im = image_cp * 0.9 + image * 0.1
    #             out_im = out_im.astype('uint8')
    #             out.write(out_im)
            count += 1
        success, image1 = cap.read()
        frame_time1 = custom_time(cap.get(cv2.CAP_PROP_POS_MSEC))
        success, image2 = cap.read()
        frame_time2 = custom_time(cap.get(cv2.CAP_PROP_POS_MSEC))
        success, image3 = cap.read()
        frame_time3 = custom_time(cap.get(cv2.CAP_PROP_POS_MSEC))
        pbar.n = count
        pbar.last_print_n = count
        pbar.refresh()

    predictions = np.array(predictions)
    np.savez_compressed(outName, predictions)
    
    end = time.time()
    print('Prediction time:', end-start, 'secs')
    print('Done......')

In [21]:
import os
matches = list('match' + str(i) for i in range(1, 27))

for match in matches:
    for video in os.listdir('/home/code-base/scratch_space/data/%s/rally_video/' % match):
        rally, _ = os.path.splitext(video)
        print(video)
        if '.mp4' not in video:
            continue
        predict_heatmaps(
            '/home/code-base/scratch_space/data/%s/rally_video/%s' % (match, video), 
            '/home/code-base/scratch_space/data/%s/ball_trajectory/%s.npz' % (match, rally)
        )

1_02_00.mp4
Beginning predicting......
About to begin prediction...


  0%|          | 0/500 [00:00<?, ?it/s]

Prediction time: 24.952389240264893 secs
Done......
1_02_04.mp4
Beginning predicting......
About to begin prediction...


  0%|          | 0/154 [00:00<?, ?it/s]

Prediction time: 7.95392918586731 secs
Done......
1_06_08.mp4
Beginning predicting......
About to begin prediction...


...

Prediction time: 25.995281457901 secs
Done......
1_02_03.mp4
Beginning predicting......
About to begin prediction...


  0%|          | 0/380 [00:00<?, ?it/s]

Prediction time: 19.23523187637329 secs
Done......
1_00_01.mp4
Beginning predicting......
About to begin prediction...


  0%|          | 0/225 [00:00<?, ?it/s]

Prediction time: 11.171113729476929 secs
Done......


In [13]:
predict_heatmaps(
    '/home/code-base/user_space/ai-badminton/notebooks/data/anthony-ginting-beat-kento-momota-anthony-ginting-vs-kento-momota-fo-2019-hlzrivktbyo.mp4',
    '/home/code-base/user_space/ai-badminton/notebooks/data/anthony-ginting-beat-kento-momota-anthony-ginting-vs-kento-momota-fo-2019-hlzrivktbyo_heatmap.npz'
)

Beginning predicting......
About to begin prediction...


  0%|          | 0/12881 [00:00<?, ?it/s]

Prediction time: 643.7390632629395 secs
Done......
